In [27]:
#Import important libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from scipy import stats
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge  # noqa F401
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline
import warnings
warnings.filterwarnings('ignore')


from scipy import stats
from scipy.stats import norm, skew

In [2]:
#importing the data into dataframes
train_data = pd.read_csv('train_house.csv',low_memory=False)
test_data = pd.read_csv('test_house.csv',low_memory=False)

In [3]:
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

## Data Exploration

In [5]:
#
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [6]:
def wrangle(datapath):
    #read data into dataframe
    df = pd.read_csv(datapath)
    #get list of features with above 20% missing values
    mask = df.isna().sum()[df.isna().sum()/len(df) > 0.2].keys().tolist()
    #drop the features above
    df.drop(columns=mask,inplace=True)
    

    #trimming the bottom and top 10% of properties in terms of "surface_covered_in_m2"
    low, high = df["MSSubClass"].quantile([0.1, 0.9])
    mask_area = df["MSSubClass"].between(low, high)
    df = df[mask_area] 
    
    #group unique feature values in 3 classes (multicollinerity columns)
    mid_unique_values =[]
    large_unique_values =[]
    target = []
    #concatinating features with mid and low counts of unique values
    for col in df.columns:
        value = df[col].nunique()
        if value < 25:
                mid_unique_values.append(col)
        elif col == "SalePrice":
            target.append(col)
        else:
                large_unique_values.append(col)
                
    data = pd.concat([df[mid_unique_values],df[target]],axis=1)
    return data

In [7]:
train = wrangle('train_house.csv')
test = wrangle('test_house.csv')

In [8]:
train.head()

,MSSubClass,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageCond,PavedDrive,3SsnPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,TA,Y,0,0,0,2,2008,WD,Normal,208500
1,20,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,TA,Y,0,0,0,5,2007,WD,Normal,181500
2,60,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,TA,Y,0,0,0,9,2008,WD,Normal,223500
3,70,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,TA,Y,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,TA,Y,0,0,0,12,2008,WD,Normal,250000


In [9]:
test.head()

,MSSubClass,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageCars,GarageQual,GarageCond,PavedDrive,3SsnPorch,PoolArea,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,...,1.0,TA,TA,Y,0,0,6,2010,WD,Normal
1,20,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,...,1.0,TA,TA,Y,0,0,6,2010,WD,Normal
2,60,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,...,2.0,TA,TA,Y,0,0,3,2010,WD,Normal
3,60,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,...,2.0,TA,TA,Y,0,0,6,2010,WD,Normal
4,120,RL,Pave,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,...,2.0,TA,TA,Y,0,0,1,2010,WD,Normal


In [10]:
print(train.shape)
print(train.info())
print(train.columns.nunique())

(1357, 57)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1357 entries, 0 to 1459
Data columns (total 57 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MSSubClass     1357 non-null   int64 
 1   MSZoning       1357 non-null   object
 2   Street         1357 non-null   object
 3   LotShape       1357 non-null   object
 4   LandContour    1357 non-null   object
 5   Utilities      1357 non-null   object
 6   LotConfig      1357 non-null   object
 7   LandSlope      1357 non-null   object
 8   Neighborhood   1357 non-null   object
 9   Condition1     1357 non-null   object
 10  Condition2     1357 non-null   object
 11  BldgType       1357 non-null   object
 12  HouseStyle     1357 non-null   object
 13  OverallQual    1357 non-null   int64 
 14  OverallCond    1357 non-null   int64 
 15  RoofStyle      1357 non-null   object
 16  RoofMatl       1357 non-null   object
 17  Exterior1st    1357 non-null   object
 18  Exterior2nd    13

In [11]:
print(test.shape)
print(test.info())
print(test.columns.nunique())

(1355, 55)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1355 entries, 0 to 1458
Data columns (total 55 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1355 non-null   int64  
 1   MSZoning       1351 non-null   object 
 2   Street         1355 non-null   object 
 3   LotShape       1355 non-null   object 
 4   LandContour    1355 non-null   object 
 5   Utilities      1353 non-null   object 
 6   LotConfig      1355 non-null   object 
 7   LandSlope      1355 non-null   object 
 8   Neighborhood   1355 non-null   object 
 9   Condition1     1355 non-null   object 
 10  Condition2     1355 non-null   object 
 11  BldgType       1355 non-null   object 
 12  HouseStyle     1355 non-null   object 
 13  OverallQual    1355 non-null   int64  
 14  OverallCond    1355 non-null   int64  
 15  RoofStyle      1355 non-null   object 
 16  RoofMatl       1355 non-null   object 
 17  Exterior1st    1354 non-null   object 
 1

In [13]:
#recasting float types in test as ints
y_train

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1357, dtype: int64

In [35]:
#drop features not common to both train and test datasets
off = []
for col in train.columns:
      if col not in test.columns:
            off.append(col)
train.drop(columns=off,inplace=True)            

In [36]:
off

['MiscVal']

In [15]:
categorical_data = train.select_dtypes(include='object').copy()
#for feature in features:
count = [len(train[features].unique()) for features in categorical_data]
data_tuples = list(zip(categorical_data,count))
datas = pd.DataFrame(data_tuples, columns=['Features','Number of distinct values'])
datas

,Features,Number of distinct values
0,MSZoning,5
1,Street,2
2,LotShape,4
3,LandContour,4
4,Utilities,2
5,LotConfig,5
6,LandSlope,3
7,Neighborhood,23
8,Condition1,9
9,Condition2,7


In [16]:
int_data = train.select_dtypes(include='int64').copy()
#for feature in features:
count = [len(train[features].unique()) for features in int_data]
data_tuple = list(zip(int_data,count))
dat = pd.DataFrame(data_tuple, columns=['Features','Number of distinct values'])
dat

,Features,Number of distinct values
0,MSSubClass,12
1,OverallQual,10
2,OverallCond,9
3,LowQualFinSF,21
4,BsmtFullBath,4
5,BsmtHalfBath,3
6,FullBath,4
7,HalfBath,3
8,BedroomAbvGr,7
9,KitchenAbvGr,4


In [17]:
#concatinating features with mid and low counts of unique values
large_unique_value =[]
for col in categorical_data.columns:
    value = len(categorical_data[col].unique())
    if value > 5:
        large_unique_value.append(col)
print(large_unique_value)

['Neighborhood', 'Condition1', 'Condition2', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'Foundation', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'Electrical', 'Functional', 'GarageType', 'GarageQual', 'GarageCond', 'SaleType', 'SaleCondition']


In [18]:
#caste of high cordinality features
for val in large_unique_value:
    top_5 = train[val].value_counts().head(5)
    train[val] = train[val].apply(lambda c: c if c in top_5 else "Others")
    

In [19]:
categorical_data = train.select_dtypes(include='object').copy()
#for feature in features:
count = [len(train[features].unique()) for features in categorical_data]
data_tuples = list(zip(categorical_data,count))
datas = pd.DataFrame(data_tuples, columns=['Features','Number of distinct values'])
datas

,Features,Number of distinct values
0,MSZoning,5
1,Street,2
2,LotShape,4
3,LandContour,4
4,Utilities,2
5,LotConfig,5
6,LandSlope,3
7,Neighborhood,6
8,Condition1,6
9,Condition2,6


In [20]:
#concatinating features with mid and low counts of unique values
large_unique_int =[]
for col in int_data.columns:
    value = len(int_data[col].unique())
    if value > 5:
        large_unique_int.append(col)
print(large_unique_int)

['MSSubClass', 'OverallQual', 'OverallCond', 'LowQualFinSF', 'BedroomAbvGr', 'TotRmsAbvGrd', '3SsnPorch', 'PoolArea', 'MiscVal', 'MoSold']


In [21]:
#caste of high cordinality features
for val in large_unique_int:
    top_5 = train[val].value_counts().head(5)
    train[val] = train[val].apply(lambda c: c if c in top_5 else 0)

In [22]:
#Statistical outlook of the numerical features
train.describe()

,MSSubClass,OverallQual,OverallCond,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,3SsnPorch,PoolArea,MiscVal,MoSold,YrSold
count,1357.000000,1357.000000,1357.000000,1357.000000,1357.000000,1357.000000,1357.000000,1357.000000,1357.000000,1357.000000,1357.000000,1357.000000,1357.000000,1357.000000,1357.000000,1357.000000,1357.000000,1357.000000
mean,35.644805,5.667649,5.386883,1.587325,0.428150,0.059690,1.563744,0.370671,2.850405,1.036846,5.511422,0.636699,1.792189,1.167281,1.522476,9.579956,3.892410,2007.819455
std,30.258971,1.849641,1.413141,26.502420,0.519704,0.243144,0.555036,0.501144,0.792960,0.196123,2.248742,0.647147,0.739428,14.431381,28.048427,68.729841,3.041295,1.322253
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2006.000000
25%,20.000000,5.000000,5.000000,0.000000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2007.000000
50%,20.000000,6.000000,5.000000,0.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,2.000000,0.000000,0.000000,0.000000,5.000000,2008.000000
75%,60.000000,7.000000,6.000000,0.000000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2.000000,0.000000,0.000000,0.000000,6.000000,2009.000000
max,120.000000,8.000000,8.000000,528.000000,3.000000,2.000000,3.000000,2.000000,5.000000,3.000000,8.000000,3.000000,4.000000,216.000000,555.000000,700.000000,8.000000,2010.000000


In [23]:
#getting columns with missing values
missing_col = train.isna().sum()[train.isna().sum() != 0].index.tolist()

In [24]:
 #fill NAN with mean value.
for col in missing_col:
    train[col] = train[col].fillna(value="mean")
    

In [25]:
#Using label_encoder for all categorical features
train_copy = train.copy()
for c in categorical_data:
    label_encoder = LabelEncoder() 
    label_encoder.fit(list(train_copy[c].values)) 
    train_copy[c] = label_encoder.transform(list(train_copy[c].values))
    
train=train_copy.copy()
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1357 entries, 0 to 1459
Data columns (total 56 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   MSSubClass     1357 non-null   int64
 1   MSZoning       1357 non-null   int32
 2   Street         1357 non-null   int32
 3   LotShape       1357 non-null   int32
 4   LandContour    1357 non-null   int32
 5   Utilities      1357 non-null   int32
 6   LotConfig      1357 non-null   int32
 7   LandSlope      1357 non-null   int32
 8   Neighborhood   1357 non-null   int32
 9   Condition1     1357 non-null   int32
 10  Condition2     1357 non-null   int32
 11  BldgType       1357 non-null   int32
 12  HouseStyle     1357 non-null   int32
 13  OverallQual    1357 non-null   int64
 14  OverallCond    1357 non-null   int64
 15  RoofStyle      1357 non-null   int32
 16  RoofMatl       1357 non-null   int32
 17  Exterior1st    1357 non-null   int32
 18  Exterior2nd    1357 non-null   int32
 19  MasVnr

In [12]:
target_col = "SalePrice"
y_train = train[target_col]
train = train.drop(columns=target_col)

In [33]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1357 entries, 0 to 1459
Data columns (total 56 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   MSSubClass     1357 non-null   int64
 1   MSZoning       1357 non-null   int32
 2   Street         1357 non-null   int32
 3   LotShape       1357 non-null   int32
 4   LandContour    1357 non-null   int32
 5   Utilities      1357 non-null   int32
 6   LotConfig      1357 non-null   int32
 7   LandSlope      1357 non-null   int32
 8   Neighborhood   1357 non-null   int32
 9   Condition1     1357 non-null   int32
 10  Condition2     1357 non-null   int32
 11  BldgType       1357 non-null   int32
 12  HouseStyle     1357 non-null   int32
 13  OverallQual    1357 non-null   int64
 14  OverallCond    1357 non-null   int64
 15  RoofStyle      1357 non-null   int32
 16  RoofMatl       1357 non-null   int32
 17  Exterior1st    1357 non-null   int32
 18  Exterior2nd    1357 non-null   int32
 19  MasVnr

In [47]:
# Build Model
model = make_pipeline(
    LabelEncoder(),
    SimpleImputer(),
    Ridge()
)
# Fit model
model.fit(train,y_train)

TypeError: fit_transform() takes 2 positional arguments but 3 were given

In [44]:
#Using label_encoder for all categorical features
test_copy = test.copy()
for c in categorical_data:
    label_encoder = LabelEncoder() 
    label_encoder.fit(list(test_copy[c].values)) 
    test_copy[c] = label_encoder.transform(list(test_copy[c].values))
    
test=test_copy.copy().dropna()
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1352 entries, 0 to 1458
Data columns (total 55 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1352 non-null   int64  
 1   MSZoning       1352 non-null   int64  
 2   Street         1352 non-null   int64  
 3   LotShape       1352 non-null   int64  
 4   LandContour    1352 non-null   int64  
 5   Utilities      1352 non-null   int64  
 6   LotConfig      1352 non-null   int64  
 7   LandSlope      1352 non-null   int64  
 8   Neighborhood   1352 non-null   int64  
 9   Condition1     1352 non-null   int64  
 10  Condition2     1352 non-null   int64  
 11  BldgType       1352 non-null   int64  
 12  HouseStyle     1352 non-null   int64  
 13  OverallQual    1352 non-null   int64  
 14  OverallCond    1352 non-null   int64  
 15  RoofStyle      1352 non-null   int64  
 16  RoofMatl       1352 non-null   int64  
 17  Exterior1st    1352 non-null   int64  
 18  Exterior

In [45]:
y_test_pred = pd.Series(model.predict(test))
y_test_pred.head()

0    107419.559434
1    162939.890762
2    184479.116107
3    188608.416038
4    223834.690852
dtype: float64

In [46]:
coefficients = model["ridge"].coef_
features = train.index
feat_imp = pd.Series(coefficients,index=features).sort_values(key=abs)
feat_imp

TypeError: 'Ridge' object is not subscriptable